In [30]:
import pandas as pd
import numpy as np
import nltk
import multiprocessing
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from catboost import CatBoostRegressor

Скачаем данные, посмотрим, содержат ли они пропуски.

In [2]:
data = pd.read_csv("./nlp/train.csv", encoding="cp1252")
data.head()

,Id,Hotel_name,Review_Title,Review_Text,Rating
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0


In [3]:
data.isna().mean(axis=0)

Id              0.00000
Hotel_name      0.00000
Review_Title    0.09145
Review_Text     0.00000
Rating          0.00000
dtype: float64

Для начала просто воспользуемся CountVectorizer и обучим линейную регрессию с параметрами по-умолчанию.

In [4]:
texts = data.Review_Text
X = CountVectorizer().fit_transform(texts)
y = data.Rating

In [5]:
linreg = LinearRegression()
linreg.fit(X, y)
pred = linreg.predict(X)
print((-np.mean(cross_val_score(LinearRegression(), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5)

134.9622019299266


In [6]:
np.std(y, ddof=1)

21.114002971450166

Видим, что результат крайне плачевный отклонение предсказанных значений от реальных в разы выше, чем стандартное отклонение, то есть среднее - более точное предсказание, чем результат модели.

Попробуем исправить ситуацию, для начала уберем из текста все, что не является словами английского языка с помощью регулярных выражений, затем уберем стоп-слова и, наконец, воспользуемся стеммингом

In [7]:
texts_san = texts.apply(lambda x: re.findall(r"\b[A-Za-z]+\b", x))

In [8]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
texts_reduced = texts_san.apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package stopwords to /home/dmitry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
stemmer = SnowballStemmer(language="english")
texts_stemmed = texts_reduced.apply(lambda x: list(map(lambda y: stemmer.stem(y), x)))

In [10]:
texts_final = texts_stemmed.str.join(" ")

In [11]:
X = CountVectorizer().fit_transform(texts_final)
print((-np.mean(cross_val_score(LinearRegression(), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5)

143.90195678152136


Ситуация стала хуже, попробуем не использовать стемминг, а остановится на выкидывании стоп-слов.

In [12]:
X = CountVectorizer().fit_transform(texts_reduced.str.join(" "))
print((-np.mean(cross_val_score(LinearRegression(), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5)

79.33542240176071


Так уже лучше, остановимся на этом варианте.

In [13]:
texts_final = texts_reduced.str.join(" ")

Попробуем воспользоваться более сложным способом векторизации: TF-IDF.

In [14]:
X = TfidfVectorizer().fit_transform(texts_final)
print((-np.mean(cross_val_score(LinearRegression(), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5)

24.496089315342655


Результат стал сильно лучше, попробуем перебрать число н-грамм.

In [15]:
best_score = np.inf
best_ngrams = tuple()
ngrams_list = [(j,i) for i in range(1, 7) for j in range(1, i+1)]
for ngrams in ngrams_list:
    X = TfidfVectorizer(ngram_range=ngrams).fit_transform(texts_final)
    score = (-np.mean(cross_val_score(LinearRegression(), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5
    if best_score > score:
        best_score = score
        best_ngrams = ngrams
print(f"Наилучший RMSE: {best_score} достигнут на н-граммах: от {best_ngrams[0]} до {best_ngrams[1]}.")

Наилучший RMSE: 14.183353697236678 достигнут на н-граммах: от 1 до 2.


Стало еще лучше, попробуем использовать регрессю с L1 регуляризацией с параметрами по-умолчанию.

In [15]:
X = TfidfVectorizer(ngram_range=(1, 2)).fit_transform(texts_final)
print((-np.mean(cross_val_score(Lasso(), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5)

21.114376226361674


Стало хуже, попробуем перебрать коэффициент регуляризации.

In [27]:
best_score = np.inf
best_ngrams = 0
alphas = np.concatenate((np.arange(0.1, 1.7, 0.2), np.arange(2, 10, 1)))
for alpha in alphas:
    score = (-np.mean(cross_val_score(Lasso(alpha=alpha), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5
    if best_score > score:
        best_score = score
        best_alpha = alpha
print(f"Наилучший RMSE: {best_score} достигнут с коэффициентом: {best_alpha}.")

Наилучший RMSE: 20.04065699882025 достигнут с коэффициентом: 0.1.


Видно, что регуляризация не помогла улучшить результат, попробуем аналогичную процедуру для L2.

In [16]:
print((-np.mean(cross_val_score(Ridge(), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5)

14.719073660324904


In [29]:
best_score = np.inf
best_ngrams = 0
alphas = np.concatenate((np.arange(0.1, 1.7, 0.2), np.arange(2, 10, 1)))
for alpha in alphas:
    score = (-np.mean(cross_val_score(Ridge(alpha=alpha), X, y, scoring="neg_mean_squared_error", cv=5)))**0.5
    if best_score > score:
        best_score = score
        best_alpha = alpha
print(f"Наилучший RMSE: {best_score} достигнут с коэффициентом: {best_alpha}.")

Наилучший RMSE: 13.910197342475541 достигнут с коэффициентом: 0.1.


Ridge регрессия справилась лучше, результат чуть лучше, чем у простой линейной регрессии.

Попробуем более сложную модель - градиентный бустинг.

In [19]:
boost = CatBoostRegressor(iterations=300, loss_function="RMSE", verbose=False, task_type="GPU")
print((-np.mean(cross_val_score(boost, X, y, scoring="neg_mean_squared_error", cv=3))) ** 0.5)

15.516067831884133


In [21]:
boost = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7, iterations=200, learning_rate=0.2, verbose=False)
print((-np.mean(cross_val_score(boost, X, y, scoring="neg_mean_squared_error", cv=3))) ** 0.5)

15.152900396322352


Путем ручного перебора параметров (в связи с ОЧЕНЬ большим временем работы GridSearch) удалось немного повысить качество, однако оно хуже, чем у Ridge регрессии.

Попробуем случайный лес

In [32]:
workers = multiprocessing.cpu_count() - 1
forest = RandomForestRegressor(n_jobs=workers)
print((-np.mean(cross_val_score(forest, X, y, scoring="neg_mean_squared_error", cv=3))) ** 0.5)

15.201563564853476


In [35]:
forest = RandomForestRegressor(random_state=42, n_jobs=workers)
parameters = {"n_estimators": [100, 200], "max_depth": [5, 10, 20], "max_features": [0.2, 0.333, "sqrt", "auto"], "max_samples": [0.1, 0.2, 0.5, None]}
gs_forest = GridSearchCV(forest, param_grid=parameters, scoring="neg_mean_squared_error", cv=3)
gs_forest.fit(X, y)
gs_forest.best_params_

{'max_depth': 20,
 'max_features': 0.2,
 'max_samples': None,
 'n_estimators': 200}

In [36]:
forest = RandomForestRegressor(max_depth=20, max_features=0.2, max_samples=None,
                               n_estimators=200, random_state=42, n_jobs=workers)
print((-np.mean(cross_val_score(forest, X, y, scoring="neg_mean_squared_error", cv=5))) ** 0.5)

15.644247254387942


При подборе параметров, стало хуже, лес с параметрвами по-умолчанию справляется хуже регресси.

Таким образом наилучшая модель - Ridge регрессия с коэффициентом регуляризации 0.1.

Получим с ее помощью предсказания для тестового набора данных.

In [58]:
data_test = pd.read_csv("./nlp/test.csv", encoding="cp1252")

In [59]:
data_test.isna().mean(axis=0)

Id              0.000000
Hotel_name      0.000000
Review_Title    0.088861
Review_Text     0.000000
dtype: float64

In [53]:
test = data_test.Review_Text
test_san = test.apply(lambda x: re.findall(r"\b[A-Za-z]+\b", x))

In [54]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
test_reduced = test_san.apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package stopwords to /home/dmitry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
vectorizer =  TfidfVectorizer(ngram_range=(1, 2)).fit(texts_final)
X = vectorizer.transform(texts_final)
X_test = vectorizer.transform(test_reduced.str.join(" "))

In [56]:
ridge = Ridge(alpha=0.1)
ridge.fit(X, y)
prediction = ridge.predict(X_test)

In [61]:
result = pd.DataFrame({"Id": data_test.Id, "Rating": prediction})

In [64]:
result.to_csv("prediction.csv", index=False)